In [ ]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

In [2]:
%pip install scikit-learn

  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 59.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.0/35.0 MB 82.5 MB/s  0:00:00m0:00:0100:01
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [scikit-learn] [scikit-learn]
Note: you may need to restart the kernel to use updated packages.


In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased")

In [1]:
#dividing datasets
import os
import random
from sklearn.model_selection import train_test_split

# Set random seed for reproducibility
random.seed(42)

# Get list of all wiki files
wiki_dir_1 = "wikipedia_data/1of2"
wiki_dir_2 = "wikipedia_data/2of2"

wiki_files_1 = [os.path.join(wiki_dir_1, f) for f in os.listdir(wiki_dir_1) if f.startswith("wiki_")]
wiki_files_2 = [os.path.join(wiki_dir_2, f) for f in os.listdir(wiki_dir_2) if f.startswith("wiki_")]

# Combine all wiki files
all_wiki_files = wiki_files_1 + wiki_files_2

# Shuffle files and split (95% train, 5% eval)
train_files, eval_files = train_test_split(all_wiki_files, test_size=0.05, random_state=42)

print(f"Train files: {len(train_files)}")
print(f"Eval files: {len(eval_files)}")

# Function to read all content from a list of files
def read_files(file_list):
    texts = []
    for file_path in file_list:
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read()
                # Split by double newlines to separate articles
                articles = content.split('\n\n')
                # Filter out empty articles
                articles = [article.strip() for article in articles if article.strip()]
                texts.extend(articles)
        except Exception as e:
            print(f"Error reading {file_path}: {e}")
    return texts

# Read train and eval data
train_texts = read_files(train_files)
eval_texts = read_files(eval_files)

print(f"Train texts: {len(train_texts)}")
print(f"Eval texts: {len(eval_texts)}")

# Save train and eval datasets to text files
with open('train_dataset.txt', 'w', encoding='utf-8') as f:
    for text in train_texts:
        f.write(text + '\n\n')

with open('eval_dataset.txt', 'w', encoding='utf-8') as f:
    for text in eval_texts:
        f.write(text + '\n\n')

print("Datasets saved to 'train_dataset.txt' and 'eval_dataset.txt'")

# Create simple dataset objects (you may need to adapt this based on your specific requirements)
class TextDataset:
    def __init__(self, texts):
        self.texts = texts
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        return self.texts[idx]

train_dataset = TextDataset(train_texts)
eval_dataset = TextDataset(eval_texts)

Train files: 162
Eval files: 9
Train texts: 882759
Eval texts: 50132
Datasets saved to 'train_dataset.txt' and 'eval_dataset.txt'


In [ ]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total # of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset= eval_dataset            # evaluation dataset
)

In [ ]:
#trainer.train()
#trainer.evaluate()